<a href="https://colab.research.google.com/github/CUSPADS2022IBX/IBXRidership/blob/main/PLUTO_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PLUTO Cleaning

In [ ]:
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=77c3c3afb6f9d5a8dc06271fa56f9bcac131bc54c5475c90fa57061a875f7c1d
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 45.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f45c3becd8cd13e5ec9377dae73c708ceabbc3332e64eed86cfc87cb123082b0
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import IPython
import wget
%matplotlib inline
IPython.display.set_matplotlib_formats('svg')
pd.plotting.register_matplotlib_converters()
sns.set_style("whitegrid")
import zipfile

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
sc = pyspark.SparkContext.getOrCreate()
spark = SparkSession(sc)
spark

#### Reading in PLUTO .csv directly from zip URL

In [ ]:
#downloading
!wget 'https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_21v4_csv.zip'
!unzip 'nyc_pluto_21v4_csv.zip'

--2022-04-09 20:50:46--  https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_21v4_csv.zip
Resolving www1.nyc.gov (www1.nyc.gov)... 23.11.9.240, 2600:1407:3c00:1086::1500, 2600:1407:3c00:109b::1500
Connecting to www1.nyc.gov (www1.nyc.gov)|23.11.9.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93698884 (89M) [application/zip]
Saving to: ‘nyc_pluto_21v4_csv.zip’

nyc_pluto_21v4_csv. 100%[===================>]  89.36M   114MB/s    in 0.8s    

2022-04-09 20:50:47 (114 MB/s) - ‘nyc_pluto_21v4_csv.zip’ saved [93698884/93698884]



In [ ]:
#reading as spark dataframe 
pluto = spark.read.csv('/content/pluto_21v4.csv',header=True)\
.select('BCT2020','lotarea', 'bldgarea', 'comarea', 'resarea',
       'officearea', 'retailarea', 'garagearea', 'strgearea', 'factryarea',
       'otherarea','numbldgs','numfloors','unitsres','unitstotal','assesstot')


In [ ]:
#list of column names for iterating 
cols = pluto.columns[1:]

#iterating through spark DF columns to cast to float 
for col_name in cols:
    pluto = pluto.withColumn(col_name, F.col(col_name).cast('float'))

#sums of counts as groupby
pluto_tract = pluto.groupBy('BCT2020').sum()

#list of column names for iterating
cols1 = pluto_tract.columns[1:]

for i in range(len(cols1)):
    pluto_tract = pluto_tract.withColumnRenamed(cols1[i],cols[i])

In [ ]:
pluto_tract.show(5)


#aggregated by station 

+-------+---------+---------+--------+---------+----------+----------+----------+---------+----------+---------+--------+------------------+--------+----------+------------+
|BCT2020|  lotarea| bldgarea| comarea|  resarea|officearea|retailarea|garagearea|strgearea|factryarea|otherarea|numbldgs|         numfloors|unitsres|unitstotal|   assesstot|
+-------+---------+---------+--------+---------+----------+----------+----------+---------+----------+---------+--------+------------------+--------+----------+------------+
|3056600|1258182.0|1254905.0|196353.0| 978845.0|   82590.0|   69635.0|    5550.0|      0.0|       0.0|  38578.0|   575.0|             762.0|   988.0|    1032.0| 5.2304088E7|
|4072300|3602899.0|1899353.0|125662.0|1756701.0|    6791.0|    8100.0|       0.0|      0.0|       0.0| 110771.0|  1045.0|1574.5899999141693|   913.0|     934.0|  9.459345E7|
|4027200|1194838.0| 644470.0| 99013.0| 538597.0|    8594.0|   49467.0|       0.0|   2550.0|       0.0|  36853.0|   617.0| 954.6699

In [ ]:
#could group by census block (~350,169) different groups 
#could aggregate by census tract (~5,000)
#could aggregate by zip (~1700)
#could aggregate by tax lots or tax blocks 